In [1]:
# shift - option - o to sort imports
import sys

import pandas as pd

sys.path.append('../')
import requests
from IPython.display import display_markdown

from datasets.compustat import COMPUSTAT as CSTAT
from datasets.crsp_data import CRSP
from gpt.GPT import GPT

### Use synthetic control and diff in diff and interrupted time series
For synthetic control:
- Create a counterfactual company based on observables

For diff in diff:
- Identify a single pair company

For interrupted TS:
- Compute change in excess return

### Other Ideas:
1. WLS based on confidence that the firm is correctly identified?
2. WLS based on confidence that the CEO death was truly unexpected?

<b> Consideration </b>: Excess returns or change in absolute returns? 

Consideration: Look at how well the CEO is performing
- Index of CEO performance? How to measure how well the CEOs are doing?

Matching looks like a reasonable approach

In [2]:
ceo_deaths = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-27/departures.csv')
ceo_deaths = ceo_deaths[ceo_deaths['departure_code'] == 1]
ceo_deaths.dropna(how='any', inplace=True, axis=1)
ceo_deaths = ceo_deaths[['gvkey', 'fyear', 'exec_fullname', 'sources']]
ceo_deaths.head()

,gvkey,fyear,exec_fullname,sources
169,3144,1997,Roberto C. Goizueta,https://www.washingtonpost.com/archive/local/1...
218,3734,2003,Joseph M. Magliochetti,https://www.toledoblade.com/frontpage/2003/09/...
316,5151,1995,Israel Cohen,https://www.baltimoresun.com/news/bs-xpm-1995-...
440,7154,2003,James R. Cantalupo,https://money.cnn.com/2004/04/19/news/fortune5...
525,8151,2000,Larry W. Brummett,https://newsok.com/article/2709352/chief-execu...


In [3]:
crsp = CRSP()
crsp.df.head()

LOADING CLEAN FILE


,permno,date,cusip,shrcd,exchcd,ticker,shrout,vol,prc,ret
0,10000,1986-02-28,68391610,10.0,3.0,OMFGA,3680.0,828.0,3.25000,-0.257143
1,10000,1986-03-31,68391610,10.0,3.0,OMFGA,3680.0,1078.0,4.43750,0.365385
2,10000,1986-04-30,68391610,10.0,3.0,OMFGA,3793.0,957.0,4.00000,-0.098592
3,10000,1986-05-30,68391610,10.0,3.0,OMFGA,3793.0,1074.0,3.10938,-0.222656
4,10000,1986-06-30,68391610,10.0,3.0,OMFGA,3793.0,1069.0,3.09375,-0.005025


In [4]:
cstat = CSTAT()
cstat.df.head()

LOADING CLEAN FILE


,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1000,1966-03-31,1966,1.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
1,1000,1966-06-30,1966,2.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
2,1000,1966-09-30,1966,3.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
3,1000,1966-12-31,1966,4.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
4,1000,1967-03-31,1967,1.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None


In [5]:
linking_df = cstat.df[['gvkey', 'cusip', 'fyearq']].drop_duplicates()
ceo_deaths = pd.merge(ceo_deaths, linking_df, left_on=['gvkey', 'fyear'], right_on=['gvkey', 'fyearq'], how='left')
ceo_deaths.drop('fyearq', axis=1, inplace=True)
ceo_deaths.head()

,gvkey,fyear,exec_fullname,sources,cusip
0,3144,1997,Roberto C. Goizueta,https://www.washingtonpost.com/archive/local/1...,191216100
1,3734,2003,Joseph M. Magliochetti,https://www.toledoblade.com/frontpage/2003/09/...,235825205
2,5151,1995,Israel Cohen,https://www.baltimoresun.com/news/bs-xpm-1995-...,374478105
3,7154,2003,James R. Cantalupo,https://money.cnn.com/2004/04/19/news/fortune5...,580135101
4,8151,2000,Larry W. Brummett,https://newsok.com/article/2709352/chief-execu...,682680103


### Summary Statistics

In [6]:
print('CEO Deaths:')
round(ceo_deaths.describe(),1).loc[['count', 'min', 'max']]

CEO Deaths:


,gvkey,fyear
count,84.0,84.0
min,1632.0,1992.0
max,186336.0,2017.0


In [7]:
cols = [
    'cusip',
    'datadate',
    'ibcy',
    'cshoq',
    'prccq',
    'atq',
    'ltq',
    'ceqq',
    'cheq',
    'niq',
    'saleq',
    'cogsq',
    'xsgaq',
    'xrdq',
    'xoprq',
    'dpq',
    'dlttq',
]

c_subset = cstat.df[cols]
c_subset.head()

rename_dict = {
    'cusip': 'cusip',
    'datadate': 'datadate',
    'ibcy': 'income_before_extra_items',
    'cshoq': 'common_shares_outstanding',
    'prccq': 'price',
    'atq': 'assets',
    'ltq': 'liabilities',
    'ceqq': 'equity',
    'cheq': 'cash',
    'niq': 'net_income',
    'saleq': 'sales',
    'cogsq': 'cost_of_goods_sold',
    'xsgaq': 'selling_general_admin_expenses',
    'xrdq': 'research_development',
    'xoprq': 'operating_expenses',
    'dpq': 'depreciation',
    'dlttq': 'long_term_debt',
}

c_subset.rename(columns=rename_dict, inplace=True)
c_subset['datadate'] = pd.to_datetime(c_subset['datadate'], format='%Y-%m-%d')
c_subset = c_subset[c_subset['datadate'] >= '1992-01-01'].reset_index(drop=True)
# Remove the checksum digit from the cusip
c_subset['cusip'] = c_subset['cusip'].str[:-1]
c_subset.head()

/var/folders/np/5kt_jvc51456b29yqfk3x76c0000gn/T/ipykernel_95528/1591525348.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_subset.rename(columns=rename_dict, inplace=True)
/var/folders/np/5kt_jvc51456b29yqfk3x76c0000gn/T/ipykernel_95528/1591525348.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_subset['datadate'] = pd.to_datetime(c_subset['datadate'], format='%Y-%m-%d')


,cusip,datadate,income_before_extra_items,common_shares_outstanding,price,assets,liabilities,equity,cash,net_income,sales,cost_of_goods_sold,selling_general_admin_expenses,research_development,operating_expenses,depreciation,long_term_debt
0,00036110,1992-02-29,9.010,15.897,15.499998,396.971,199.895,197.076,2.610,2.880,104.920,81.809,14.026,NaN,95.835,3.095,67.390
1,00036110,1992-05-31,10.020,15.899,12.874998,395.351,198.614,196.737,4.197,1.010,108.450,83.133,14.824,NaN,97.957,2.783,67.323
2,00036110,1992-08-31,3.103,15.842,11.999999,398.888,199.057,199.831,5.594,3.103,98.072,76.181,12.678,NaN,88.859,2.790,66.672
3,00036110,1992-11-30,4.678,15.808,11.874999,400.203,203.736,196.467,4.890,1.575,101.930,81.781,13.042,NaN,94.823,2.832,66.606
4,00036110,1993-02-28,-1.027,15.898,12.374999,374.043,185.053,188.990,3.931,-5.705,82.336,64.760,12.720,NaN,77.480,2.831,66.365


In [8]:
crsp_df = crsp.df[crsp.df['date'] > '1992-01-01']
crsp_df['datadate'] = pd.to_datetime(crsp_df['date'])
crsp_df.head()

/var/folders/np/5kt_jvc51456b29yqfk3x76c0000gn/T/ipykernel_95528/2899558561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_df['datadate'] = pd.to_datetime(crsp_df['date'])


,permno,date,cusip,shrcd,exchcd,ticker,shrout,vol,prc,ret,datadate
88,10001,1992-01-31,36720410,11.0,3.0,GFGC,1075.0,211.0,13.750,-0.051724,1992-01-31
89,10001,1992-02-28,36720410,11.0,3.0,GFGC,1075.0,405.0,11.000,-0.200000,1992-02-28
90,10001,1992-03-31,36720410,11.0,3.0,GFGC,1075.0,282.0,11.750,0.081818,1992-03-31
91,10001,1992-04-30,36720410,11.0,3.0,GFGC,1075.0,119.0,11.875,0.010638,1992-04-30
92,10001,1992-05-29,36720410,11.0,3.0,GFGC,1075.0,137.0,12.000,0.010526,1992-05-29


In [9]:
crsp_df

,permno,date,cusip,shrcd,exchcd,ticker,shrout,vol,prc,ret,datadate
88,10001,1992-01-31,36720410,11.0,3.0,GFGC,1075.0,211.0,13.75000,-0.051724,1992-01-31
89,10001,1992-02-28,36720410,11.0,3.0,GFGC,1075.0,405.0,11.00000,-0.200000,1992-02-28
90,10001,1992-03-31,36720410,11.0,3.0,GFGC,1075.0,282.0,11.75000,0.081818,1992-03-31
91,10001,1992-04-30,36720410,11.0,3.0,GFGC,1075.0,119.0,11.87500,0.010638,1992-04-30
92,10001,1992-05-29,36720410,11.0,3.0,GFGC,1075.0,137.0,12.00000,0.010526,1992-05-29
...,...,...,...,...,...,...,...,...,...,...,...
3785681,93436,2023-08-31,88160R10,11.0,3.0,TSLA,3173994.0,25029170.0,258.07999,-0.034962,2023-08-31
3785682,93436,2023-09-29,88160R10,11.0,3.0,TSLA,3179000.0,24395440.0,250.22000,-0.030456,2023-09-29
3785683,93436,2023-10-31,88160R10,11.0,3.0,TSLA,3178921.0,25905681.0,200.84000,-0.197346,2023-10-31
3785684,93436,2023-11-30,88160R10,11.0,3.0,TSLA,3178921.0,26395792.0,240.08000,0.195379,2023-11-30


In [10]:
# Merge the datasets
merged = pd.merge(crsp_df, c_subset, on=['cusip', 'datadate'], how='left')
# Forward fill the cstat data 
merged = merged.groupby('permno').ffill()
merged.dropna(how='any', inplace=True)
merged

,date,cusip,shrcd,exchcd,ticker,shrout,vol,prc,ret,datadate,...,equity,cash,net_income,sales,cost_of_goods_sold,selling_general_admin_expenses,research_development,operating_expenses,depreciation,long_term_debt
720,1992-01-31,12709510,11.0,3.0,CBOT,7597.0,39140.0,14.00000,-0.008850,1992-01-31,...,17.814,1.246,0.696,10.847,4.871,4.318,0.493,9.189,0.298,12.465
721,1992-02-28,12709510,11.0,3.0,CBOT,7597.0,11392.0,13.37500,-0.044643,1992-02-28,...,17.814,1.246,0.696,10.847,4.871,4.318,0.493,9.189,0.298,12.465
722,1992-03-31,12709510,11.0,3.0,CBOT,7597.0,17671.0,13.25000,-0.009346,1992-03-31,...,17.814,1.246,0.696,10.847,4.871,4.318,0.493,9.189,0.298,12.465
723,1992-04-30,12709510,11.0,3.0,CBOT,7597.0,12783.0,10.75000,-0.188679,1992-04-30,...,18.528,0.726,0.680,12.564,5.838,4.922,0.591,10.760,0.184,73.741
724,1992-05-29,12709510,11.0,3.0,CBOT,7600.0,8732.0,9.12500,-0.151163,1992-05-29,...,18.528,0.726,0.680,12.564,5.838,4.922,0.591,10.760,0.184,73.741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1905492,2023-08-31,88160R10,11.0,3.0,TSLA,3173994.0,25029170.0,258.07999,-0.034962,2023-08-31,...,51130.000,23459.000,2703.000,24927.000,19240.000,2134.000,943.000,21374.000,1154.000,3803.000
1905493,2023-09-29,88160R10,11.0,3.0,TSLA,3179000.0,24395440.0,250.22000,-0.030456,2023-09-29,...,51130.000,23459.000,2703.000,24927.000,19240.000,2134.000,943.000,21374.000,1154.000,3803.000
1905494,2023-10-31,88160R10,11.0,3.0,TSLA,3178921.0,25905681.0,200.84000,-0.197346,2023-10-31,...,51130.000,23459.000,2703.000,24927.000,19240.000,2134.000,943.000,21374.000,1154.000,3803.000
1905495,2023-11-30,88160R10,11.0,3.0,TSLA,3178921.0,26395792.0,240.08000,0.195379,2023-11-30,...,51130.000,23459.000,2703.000,24927.000,19240.000,2134.000,943.000,21374.000,1154.000,3803.000


In [11]:
merged.columns

Index(['date', 'cusip', 'shrcd', 'exchcd', 'ticker', 'shrout', 'vol', 'prc',
       'ret', 'datadate', 'income_before_extra_items',
       'common_shares_outstanding', 'price', 'assets', 'liabilities', 'equity',
       'cash', 'net_income', 'sales', 'cost_of_goods_sold',
       'selling_general_admin_expenses', 'research_development',
       'operating_expenses', 'depreciation', 'long_term_debt'],
      dtype='object')

In [12]:
merged['year'] = merged['datadate'].dt.year
merged['month'] = merged['datadate'].dt.month
merged = merged.drop(['datadate', 'date', 'shrcd'], axis=1)
merged = merged.round(2)
merged

,cusip,exchcd,ticker,shrout,vol,prc,ret,income_before_extra_items,common_shares_outstanding,price,...,net_income,sales,cost_of_goods_sold,selling_general_admin_expenses,research_development,operating_expenses,depreciation,long_term_debt,year,month
720,12709510,3.0,CBOT,7597.0,39140.0,14.00,-0.01,0.70,7.6,14.00,...,0.70,10.85,4.87,4.32,0.49,9.19,0.30,12.46,1992,1
721,12709510,3.0,CBOT,7597.0,11392.0,13.38,-0.04,0.70,7.6,14.00,...,0.70,10.85,4.87,4.32,0.49,9.19,0.30,12.46,1992,2
722,12709510,3.0,CBOT,7597.0,17671.0,13.25,-0.01,0.70,7.6,14.00,...,0.70,10.85,4.87,4.32,0.49,9.19,0.30,12.46,1992,3
723,12709510,3.0,CBOT,7597.0,12783.0,10.75,-0.19,1.38,7.6,10.75,...,0.68,12.56,5.84,4.92,0.59,10.76,0.18,73.74,1992,4
724,12709510,3.0,CBOT,7600.0,8732.0,9.12,-0.15,1.38,7.6,10.75,...,0.68,12.56,5.84,4.92,0.59,10.76,0.18,73.74,1992,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1905492,88160R10,3.0,TSLA,3173994.0,25029170.0,258.08,-0.03,5153.00,3174.0,261.77,...,2703.00,24927.00,19240.00,2134.00,943.00,21374.00,1154.00,3803.00,2023,8
1905493,88160R10,3.0,TSLA,3179000.0,24395440.0,250.22,-0.03,5153.00,3174.0,261.77,...,2703.00,24927.00,19240.00,2134.00,943.00,21374.00,1154.00,3803.00,2023,9
1905494,88160R10,3.0,TSLA,3178921.0,25905681.0,200.84,-0.20,5153.00,3174.0,261.77,...,2703.00,24927.00,19240.00,2134.00,943.00,21374.00,1154.00,3803.00,2023,10
1905495,88160R10,3.0,TSLA,3178921.0,26395792.0,240.08,0.20,5153.00,3174.0,261.77,...,2703.00,24927.00,19240.00,2134.00,943.00,21374.00,1154.00,3803.00,2023,11


In [13]:
# remove checksum digit from cusip
ceo_deaths['cusip'] = ceo_deaths['cusip'].str[:-1]
ceo_deaths = ceo_deaths.drop('gvkey', axis=1)
ceo_deaths

,fyear,exec_fullname,sources,cusip
0,1997,Roberto C. Goizueta,https://www.washingtonpost.com/archive/local/1...,19121610
1,2003,Joseph M. Magliochetti,https://www.toledoblade.com/frontpage/2003/09/...,23582520
2,1995,Israel Cohen,https://www.baltimoresun.com/news/bs-xpm-1995-...,37447810
3,2003,James R. Cantalupo,https://money.cnn.com/2004/04/19/news/fortune5...,58013510
4,2000,Larry W. Brummett,https://newsok.com/article/2709352/chief-execu...,68268010
...,...,...,...,...
79,2010,Richard N. Berman,https://www.aftermarketnews.com/dorman-product...,25827810
80,2015,Joshua E. Comstock,https://www.bizjournals.com/houston/news/2016/...,12674R10
81,2011,Wendy Carlson Waugaman,http://ir.american-equity.com/news-releases/ne...,02567620
82,2016,Fred L. Callon,https://www.natchezdemocrat.com/2017/05/25/cal...,13123X50


### Instantiating GPT

In [14]:
gpt = GPT()

Instance does not exist
Creating openai instance
Key (hashed): 9bc934e24e3fb9c92d2198a5683badf0
Org: org-KOWVaa9rBNtxQgrhf0tXH5Tr
Instance exists
Initializing GPT, model type: gpt-4o


In [15]:
gpt.add_system_prompt('You will receive an article about a particular ceo that dies. Return the month and year of the article.')
gpt.add_system_prompt('Respond just with the numerical month only.')

Added system prompt: You will receive an article about a particular ceo that dies. Return the month and year of the article.
Added system prompt: Respond just with the numerical month only.


In [20]:
ceo_deaths['month'] = None
for row in ceo_deaths.iterrows():
    source = row[1]['sources']
    month = gpt.query(source)
    ceo_deaths.loc[row[0], 'month'] = month

ceo_deaths

Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT
Querying GPT

,fyear,exec_fullname,sources,cusip,month
0,1997,Roberto C. Goizueta,https://www.washingtonpost.com/archive/local/1...,19121610,10
1,2003,Joseph M. Magliochetti,https://www.toledoblade.com/frontpage/2003/09/...,23582520,09
2,1995,Israel Cohen,https://www.baltimoresun.com/news/bs-xpm-1995-...,37447810,11
3,2003,James R. Cantalupo,https://money.cnn.com/2004/04/19/news/fortune5...,58013510,04
4,2000,Larry W. Brummett,https://newsok.com/article/2709352/chief-execu...,68268010,06
...,...,...,...,...,...
79,2010,Richard N. Berman,https://www.aftermarketnews.com/dorman-product...,25827810,01
80,2015,Joshua E. Comstock,https://www.bizjournals.com/houston/news/2016/...,12674R10,03
81,2011,Wendy Carlson Waugaman,http://ir.american-equity.com/news-releases/ne...,02567620,6
82,2016,Fred L. Callon,https://www.natchezdemocrat.com/2017/05/25/cal...,13123X50,5


In [23]:
ceo_deaths['month'] = pd.to_numeric(ceo_deaths['month'], errors='coerce')
ceo_deaths.to_csv('exported/ceo_deaths.csv', index=False)

In [24]:
merged.to_csv('exported/merged.csv', index=False)